In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import registration as rg

from registration.utils import create_nifti_image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
templateImagePath = "../CCF_DATA/average_template_25.nii.gz"
dataImagePath = "brain_25.nii.gz"
annotationImagePath = "../CCF_DATA/annotation_25.nii.gz"

fixedImagePath = templateImagePath
movingImagePath = dataImagePath

In [4]:
import os
outputDir = "reg"
if not os.path.isdir(outputDir):
    os.mkdir(outputDir)
    
if not os.path.isdir("reg/tests"):
    os.mkdir("reg/tests")

In [5]:
A, axisAlignedData = rg.axisAlignData(fixedImagePath, movingImagePath)

axisAlignedDataPath  = os.path.join(outputDir , "axisAlignedData.nii.gz")
create_nifti_image(axisAlignedData, 25, axisAlignedDataPath, 1)

100%|██████████| 528/528 [00:01<00:00, 447.46it/s]


In [6]:
elastixResult  = rg.elastixRegistration(fixedImagePath , axisAlignedDataPath, outputDir, rescale=True)

In [7]:
elastixResult  = rg.elastixTransformation(axisAlignedDataPath, outputDir)

In [8]:
deformationField  = rg.sliceToSlice3DLaplacian(fixedImagePath , elastixResult , axis =0 )

 89%|████████▊ | 468/528 [05:38<00:16,  3.54it/s]c:\Users\atchu\anaconda3\envs\tissuecyte\lib\site-packages\numpy\lib\function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
100%|██████████| 528/528 [06:08<00:00,  1.43it/s]


Building data for Laplacian Sparse Matrix A
Creating Laplacian Sparse Matrix A


In [31]:
transformedData   = rg.applyDeformationField(movingImagePath , deformationField)
refinedResultPath = os.path.join(outputDir, "elastixRefined.nii.gz")
create_nifti_image(transformedData, 2.5, refinedResultPath, 1/2)
np.save(os.path.join(outputDir,"deformation3d.npy"), deformationField)